In [1]:
import sys
from run_experiment import *

In [2]:
# Test yrange_to_range

yrange_test_cases = [
    {"input": [1, 2, 3, 4], "output": [1, 2, 3, 4]},
    {"input": {"end": 4}, "output": [0, 1, 2, 3]},
    {"input": {"start": 1, "end": 4}, "output": [1, 2, 3]},
    {"input": {"start": 1, "end": 4, "step": 3}, "output": [1]},
    {"input": {"start": 1, "end": 4, "step": 1, "map": "lambda x : 2**x"}, "output": [2, 4, 8]},
    
    # error test
    {"input": {"start": 1}, "output": []},
    {"input": 1, "output": []},
]

for case in yrange_test_cases:
    assert yrange_to_range(case["input"]) == case["output"]

print("yrange_to_range passed")

yrange_to_range passed


In [3]:
conf = "conf/test_experiment.yaml"

with open(conf, 'r') as f:
    file_data = f.read()

DEFAULT_OPT_INTERVAL = 60

cfg = yaml.load(file_data, yaml.FullLoader)
# cfg = yaml.load(file_data)
if "default_opt_interval" in cfg:
    DEFAULT_OPT_INTERVAL = cfg["default_opt_interval"]

if "opt_interval" not in cfg["workload_exec"]:
    cfg["workload_exec"]["raw"]["opt_interval"] = DEFAULT_OPT_INTERVAL

if "stress_exec" in cfg and "opt_interval" not in cfg["stress_exec"]:
    cfg["stress_exec"]["raw"]["opt_interval"] = DEFAULT_OPT_INTERVAL

exp = exp_from_yaml(cfg["experiment"])
workload_exec = workload_exec_from_yaml(cfg["workload_exec"])
stress_exec = None
if "stress_exec" in cfg:
    stress_exec = stress_exec_from_yaml(cfg["stress_exec"])
exp.run(stress_exec=stress_exec, workload_exec=workload_exec, interval=0)
exp.__dict__

INFO:root:1698218539 experiment start
INFO:root:run: ssh fhl@worknode-01 sudo podman run --name stress_mem -d --rm --cpuset-cpus 40-59,125-139 localhost/my-stress-ng:v0.1 --memrate-rd-mbs 1000 --memrate-wr-mbs 1000 --memrate 1 --memrate-byte 1G
INFO:root:run: docker exec -it redis-client-1 memtier_benchmark -s envoy --test-time 10
INFO:root:run: docker exec -it redis-client-1 memtier_benchmark  -s envoy --test-time 120 -t 100
INFO:root:run: ssh fhl@worknode-01 sudo podman stop stress_mem
INFO:root:run: ssh fhl@worknode-01 sudo podman run --name stress_mem -d --rm --cpuset-cpus 40-59,125-139 localhost/my-stress-ng:v0.1 --memrate-rd-mbs 1000 --memrate-wr-mbs 1000 --memrate 2 --memrate-byte 1G
INFO:root:run: docker exec -it redis-client-1 memtier_benchmark -s envoy --test-time 10
INFO:root:run: docker exec -it redis-client-1 memtier_benchmark  -s envoy --test-time 120 -t 100
INFO:root:run: ssh fhl@worknode-01 sudo podman stop stress_mem
INFO:root:run: ssh fhl@worknode-01 sudo podman run -

{'start_time': 1698218539,
 'end_time': 1698218539,
 'total_time': 0,
 'n_epoch': 19,
 'date_format': 'timestamp',
 'info_per_workload': {'redis_0': {'info_per_epoch': [{'start_time': 1698218539,
     'run_cmd': 'docker exec -it redis-client-1 memtier_benchmark  -s envoy --test-time 120 -t 100',
     'end_time': 1698218539,
     'name': 'redis_0',
     'stress': {'mem': {'memrate': '1', 'memrate-byte': '1G'}}},
    {'start_time': 1698218539,
     'run_cmd': 'docker exec -it redis-client-1 memtier_benchmark  -s envoy --test-time 120 -t 100',
     'end_time': 1698218539,
     'name': 'redis_0',
     'stress': {'mem': {'memrate': '2', 'memrate-byte': '1G'}}},
    {'start_time': 1698218539,
     'run_cmd': 'docker exec -it redis-client-1 memtier_benchmark  -s envoy --test-time 120 -t 100',
     'end_time': 1698218539,
     'name': 'redis_0',
     'stress': {'mem': {'memrate': '3', 'memrate-byte': '1G'}}},
    {'start_time': 1698218539,
     'run_cmd': 'docker exec -it redis-client-1 memtie

In [4]:
data_root = cfg["data_root"]

dir_path = os.path.join(data_root, exp.dir_name())
print(dir_path)
# if not os.path.exists(dir_path):
#     os.mkdir(dir_path)

# with open(os.path.join(dir_path, cfg["experiment"]["save_file"]), 'w') as f:
#     json.dump(exp.__dict__, f)

# with open(os.path.join(dir_path, cfg["workload_exec"]["save_file"]), 'w') as f:
#     json.dump(workload_exec.info_per_workload, f)

/home/fhl/Workplace/Python/appProfile/data/standard_stress_cpu_20231022161723


In [5]:
# import json
# with open("/home/ict/appProfile/data/standard_stress_no_20231016091241/exp.json", "r") as f:
#     print(f.readlines())
    # exp = json.load(f)

In [6]:
# ## Collet Data
# grafana_auth = cfg["collect"]["grafana"]["auth"]
# grafana_server = cfg["collect"]["grafana"]["server"]
# prom_server = cfg["collect"]["prometheus"]["server"]
# aio_db = cfg["collect"]["query"]["dashboard"]
# step = cfg["collect"]["query"]["step"]
# dash_boards = cfg["collect"]["query"]["dashboard"]

# pclient = prometheus.client(prom_server)
# gclient = grafana.client(grafana_server, grafana_auth)

# db_datas = [gclient.get_db(db) for db in dash_boards]
# assert len(db_datas) > 0, "no prometheus data collect"

# if "aio" in dash_boards:
#     logging.warning("while aio dashboard is existing, only fetching aio data")
#     dash_boards = ["aio"]

# targets = []
# if len(dash_boards) == 1 and dash_boards[0] == "aio":
#     # assert it is aio
#     targets = db_datas[0]["panels"][0]["targets"]
# else:
#     for db_data in db_datas:
#         targets = targets + config_parser.read_targets_from_json(db_data)

# df = pclient.targets_to_df(targets, exp.start_time, exp.end_time, step)
# print(df.info())
# df.to_csv(os.path.join(dir_path, cfg["collect"]["save_file"]))